In [1]:
import abc
import inspect
from contextlib import contextmanager

In [2]:
class MyMeta(type):
    def __new__(meta, name, bases, dct):
        print '-----------------------------------'
        print "Allocating memory for class", name
        print meta
        print bases
        print dct
        return super(MyMeta, meta).__new__(meta, name, bases, dct)
    def __init__(cls, name, bases, dct):
        print '-----------------------------------'
        print "Initializing class", name
        print cls
        print bases
        print dct
        super(MyMeta, cls).__init__(name, bases, dct)

# first version - Liftable-signal

In [3]:
def use_as_needed(func, kwargs):
    meta = inspect.getargspec(func)
    if meta.keywords is not None:
            return func(**kwargs)
    else:
        # not generic super-constructor - pick only the relevant subentries:
        return func(**{k:kwargs[k] for k in kwargs if k in meta.args})

class NotLiftable(RuntimeError):
    pass

@contextmanager
def super_liftable(cls, self):
    """ this is kind of a hack to replace super.super, however I haven't found any other nice way to do it """
    if cls is object:
        raise NotLiftable()
    liftables = [l for l in cls.__bases__ if type(l).__name__ == "Liftable"]
    if not liftables:
        raise NotLiftable()
        
    orig_class = self.__class__
    self.__class__ = liftables[0]
    yield self
    self.__class__ = orig_class

    
def LiftableFrom(base_cls_name):
    
    class Liftable(type):
        def __init__(cls, name, bases, dct):
            # for base_cls nothing should be done, as this is the one to refer to by Lifting
            if not cls.__name__ == base_cls_name:
                if "__init__" in dct:
                    raise TypeError("Descendents of Liftable are not allowed to have own __init__ method. Instead overwrite __initialize__")
                
                def lifted__init__(self, **kwargs):
                    with super_liftable(cls, self) as s:
                        use_as_needed(s.__init__, kwargs)
                    if hasattr(self, "__initialize__"):
                        use_as_needed(self.__initialize__, kwargs)

                cls.__init__ = lifted__init__
                #setattr(cls, "__init__", lifted__init__)
                
            super(Liftable, cls).__init__(name, bases, dct)
    
    Liftable.base_cls_name = base_cls_name
    #Liftable.__name__ = "LiftableFrom" + base_cls_name   # to show that this is possible
    return Liftable

##  tests

In [4]:
class A(object):
    __metaclass__ = LiftableFrom("A")
    def __init__(self, a):
        self.a = a
        
class B(A):
    def __initialize__(self, b):
        print "initialize b"
        self.b = b
        
class C(B):
    def __initialize__(self, c):
        print "initialize c"
        self.c = c

In [5]:
a = A(a=1)
a.a

1

In [ ]:
b = B(a=1, b=2)
b.a, b.b

In [ ]:
type(b)

In [ ]:
c = C(a=1, b=2, c=3)
c.a, c.b, c.c

In [ ]:
lift(a, C, b=2, c=3)
print type(a)
print a.a, a.b, a.c

# second version - Lift factory

In [ ]:
@contextmanager
def mysuper(cls, self):
    orig_class = self.__class__
    self.__class__ = cls
    yield self
    self.__class__ = orig_class
    
def Lift(cls):
    """ class decorator """
    class _Lift(cls):
        __metaclass__ = abc.ABCMeta
        
        def __init__(self, **kwargs):
            print "init ", cls.__name__
            print self.__class__.__mro__
            with mysuper(cls, self) as s:
                use_as_needed(s.__init__, kwargs)
#             #TODO the following does not work, but would be the first thing to try
#             #gives TypeError: <method-wrapper '__init__' of C object at 0x7f0ee504a610> is not a Python function
#             #i.e. super(cls, self).__init__ is not an inspectable function as one would expect
#             use_as_needed(super(cls, self).__init__, kwargs) 
            use_as_needed(self.__initialize__, kwargs)
        
        @abc.abstractmethod
        def __initialize__(self, **kwargs):
            return NotImplemented()
    
    _Lift.__name__ = "_Lift_" + cls.__name__
    return _Lift

## tests

In [ ]:
class A2(object):
    def __init__(self, a):
        self.a = a
    
class B2(Lift(A2)):
    def __initialize__(self, b):
        print "initilize b"
        self.b = b

class C2(Lift(B2)):
    def __initialize__(self, c):
        print "initliaz c"
        self.c = c

In [ ]:
c2 = C2(a=1, b=2, c=3)
c2.a, c2.b, c2.c

# lift

In [ ]:
def lift(self, new_class, **kwargs): #TODO adapt to work with both definitions above
    # Stop Conditions:
    if self.__class__ is new_class:
        return # nothing to do
    elif new_class is object: # Base Case
        # break recursion at once:
        raise NotLiftable()
    
    liftables = [l for l in new_class.__bases__ if type(l).__name__ == "Liftable"]
    lifts = [l.__base__ for l in new_class.__bases__ if l.__name__.startswith("_Lift_")]
    ls = liftables + lifts
    if not ls:
        raise NotLiftable()

    # recursive case:
    if not self.__class__ is ls[0]: # it would also be possible to use tree like left-first-search here
        lift(self, ls[0], **kwargs)
    # own case:
    self.__class__ = new_class
    use_as_needed(self.__initialize__, kwargs)

In [ ]:
a = A(a=1)

In [ ]:
lift(a, B, b=1)

In [ ]:
a.b

In [ ]:
lift(a, C, c=1)

In [ ]:
a.c

In [ ]:
a2 = A2(a=1)

In [ ]:
C2.__bases__[0].__name__.startswith("_Lift_")

In [ ]:
type(a2)

In [ ]:
lift(a2, C2, b=2, c=3)

In [ ]:
a2

In [ ]:
class M(object):
    a = __class__
    def __init__(self):
        pass

In [ ]:
m = M()